# Introduction
I use this notebook to run manual administration tasks on the datastore.

In [9]:
# imports
import os
from dotenv import load_dotenv
from pymongo import MongoClient

In [10]:
from training.sm2_anki.stats import Stats
from training.sm2_anki.recall import Recall

In [11]:
# initialize environment
load_dotenv('../.env')
client = MongoClient(os.environ['MONGODB_URI'])

In [12]:
db = client.chinese
vocabulary_connection = db['vocabulary']

In [13]:
all_entries = list(vocabulary_connection.find({}))

In [14]:
for entry in all_entries:
    stats = entry['stats']
    
    for fact in stats:
        stats[fact]['interval'] = max(stats[fact]['interval'], 365)
        stats[fact]['ef'] = max(stats[fact]['ef'], 5.0)

In [15]:
entry

{'_id': ObjectId('63cc654e8b14afcdbc4d9f8e'),
 'lexeme_id': ObjectId('63cc654d8b14afcdbc4d9f8d'),
 'user_id': ObjectId('62a57d5bfa96028f59ac1d75'),
 'stats': {}}

In [16]:
vocabulary_connection.drop()
vocabulary_connection.insert_many(all_entries)